In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [2]:
from openai_hackathon_wait.tools.arxiv_tool import arxiv_search

In [3]:
PAPER_CONTENT = """
# Abstract

We present PPCEF, a novel method for generating prob-
abilistically plausible counterfactual explanations (CFs). PPCEF ad-
vances beyond existing methods by combining a probabilistic formula-
tion that leverages the data distribution with the optimization of plausi-
bility within a unified framework. Compared to reference approaches,
our method enforces plausibility by directly optimizing the explicit
density function without assuming a particular family of parametrized
distributions. This ensures CFs are not only valid (i.e., achieve class
change) but also align with the underlying data’s probability density.
For that purpose, our approach leverages normalizing flows as power-
ful density estimators to capture the complex high-dimensional data
distribution. Furthermore, we introduce a novel loss function that bal-
ances the trade-off between achieving class change and maintaining
closeness to the original instance while also incorporating a proba-
bilistic plausibility term. PPCEF’s unconstrained formulation allows
for an efficient gradient-based optimization with batch processing,
leading to orders of magnitude faster computation compared to prior
methods. Moreover, the unconstrained formulation of PPCEF allows
for the seamless integration of future constraints tailored to specific
counterfactual properties. Finally, extensive evaluations demonstrate
PPCEF’s superiority in generating high-quality, probabilistically plau-
sible counterfactual explanations in high-dimensional tabular settings.

# Keywords
Counterfactual Explanations, Probabilistic Plausibility, Normalizing Flows, Explainable AI (XAI)
"""

In [4]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
import asyncio

arxiv_agent = Agent(
    name="Arxiv agent",
    instructions="You provide information about the papers that are related to the query. Return between 15 and 25 results by default.",
    tools=[arxiv_search],
    model="gpt-4o-mini",
    output_type=list[str]
)

triage_agent = Agent(
    name="Triage agent",
    instructions="You are a helpful assistant that finds the most relevant papers from Arxiv.",
    handoffs=[arxiv_agent],
    model="gpt-4o-mini",
    output_type=list[str]
)

In [5]:
async def main():
    result = await Runner.run(
        triage_agent, input=PAPER_CONTENT
    )
    print(result.final_output)

In [6]:
res = await main()

2025-04-26 00:56:36.574 | INFO     | openai_hackathon_wait.tools.arxiv_tool:arxiv_search:60 - Max results: 20
2025-04-26 00:56:36.575 | INFO     | openai_hackathon_wait.tools.arxiv_tool:arxiv_search:61 - Sort by: relevance
2025-04-26 00:56:36.576 | INFO     | openai_hackathon_wait.tools.arxiv_tool:arxiv_search:73 - Starting arXiv search for: PPCEF, novel method for generating prob-ablistically plausible counterfactual explanations, combinin...
2025-04-26 00:56:40.283 | INFO     | openai_hackathon_wait.api.utils.get_article_keywords:get_article_keywords:42 - Article keywords response: keywords=['PPCEF', 'probabilistic counterfactual explanations', 'novel method for counterfactuals', 'probabilistic formulation', 'optimizing plausibility', 'density function', 'normalizing flows', 'counterfactual properties', 'gradient-based optimization', 'Explainable AI', 'XAI', 'explainable machine learning', 'causal inference', 'probabilistic modeling', 'counterfactual reasoning', 'machine learning int

['http://arxiv.org/pdf/2404.16174v1', 'http://arxiv.org/pdf/2403.13536v1', 'http://arxiv.org/pdf/2402.08934v1', 'http://arxiv.org/pdf/2401.05559v1', 'http://arxiv.org/pdf/2402.05860v1', 'http://arxiv.org/pdf/2402.08456v1', 'http://arxiv.org/pdf/2401.10256v1', 'http://arxiv.org/pdf/2403.02097v1', 'http://arxiv.org/pdf/2402.09048v1', 'http://arxiv.org/pdf/2401.07865v1', 'http://arxiv.org/pdf/2404.10403v1', 'http://arxiv.org/pdf/2402.09567v1', 'http://arxiv.org/pdf/2401.07365v1', 'http://arxiv.org/pdf/2403.12345v1', 'http://arxiv.org/pdf/2401.08825v1', 'http://arxiv.org/pdf/2403.15689v1', 'http://arxiv.org/pdf/2402.06789v1', 'http://arxiv.org/pdf/2402.06123v1', 'http://arxiv.org/pdf/2401.09541v1', 'http://arxiv.org/pdf/2404.13456v1', 'http://arxiv.org/pdf/2401.04561v1']


In [7]:
res